# 浜名湖4月8日 9R 結果

In [4]:
sample_html = \
    'http://boatrace.jp/owpc/pc/race/raceresult?rno=12&jcd=06&hd=20200410'

In [5]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

## htmlをsoupに取り込む

In [6]:
target_url = sample_html
__html_content = urlopen(target_url).read()
__soup = BeautifulSoup(__html_content, 'html.parser')

### 結果表を抜く

In [7]:
__target_table_selector = \
    'body > main > div > div > div > '\
    'div.contentsFrame1_inner > div.grid.is-type2.h-clear.h-mt10 > '\
    'div:nth-child(1) > div > table'
__target_table_html = __soup.select_one(__target_table_selector)
player_res_html_list = __target_table_html.select('tbody')

In [46]:
waku_dict = {}
for rank_p_html in player_res_html_list:
    rank, waku, name, time = \
        list(map(lambda x: x.text, rank_p_html.select('td')))
    # rankはF,L欠などが存在するためエラーハンドルがいる
    try:
        rank = int(rank)
    except ValueError:
        rank = -1
#     time = time.replace('"', '.')
    waku = int(waku)
    name = name.replace('\n', '').replace('\u3000', '').replace(' ', '')
    no, name = name.split('\r')
    no = int(no)

    __content_dict = {
        'rank': rank,
        'name': name,
        'time': time
    }

    waku_dict[waku] = __content_dict

In [51]:
waku_dict[1]["time"]

' '

In [52]:
from datetime import datetime, timedelta

In [56]:
t = datetime.strptime(waku_dict[6]["time"], '%M\'%S"%f')

ValueError: time data ' ' does not match format '%M\'%S"%f'

In [54]:
delta = timedelta(
    seconds=t.second,
    microseconds=t.microsecond,
    minutes=t.minute,
)

In [55]:
delta.total_seconds()

115.8